In [24]:
import numpy as np
from scipy.stats import norm
import matplotlib.pyplot as plt

Chocs gaussiens corrélés :

$$
Z_1 \sim \mathcal N(0,1), \qquad
Z_2 = \rho Z_1 + \sqrt{1-\rho^2}\,\varepsilon
$$



In [25]:
def correlated_normals(rho, size=1):
    z1 = np.random.normal(size=size)
    eps = np.random.normal(size=size)
    z2 = rho * z1 + np.sqrt(1 - rho**2) * eps
    return z1, z2


Un pas de temps du modèle SABR simplifié (Euler).

$$
\begin{aligned}
S_{t+1} &= S_t + (r-q)S_t\Delta t + \sigma_t S_t \sqrt{\Delta t}\,Z_1 \\
\sigma_{t+1} &= \sigma_t + v \sigma_t \sqrt{\Delta t}\,Z_2
\end{aligned}
$$


In [26]:
def simulate_sabr_step(S, sigma, dt, r, q, v, rho, z1, z2):
    S_next = S + (r - q) * S * dt + sigma * S * np.sqrt(dt) * z1
    sigma_next = sigma + v * sigma * np.sqrt(dt) * z2
    return S_next, sigma_next


In [27]:
def simulate_sabr_path(S0, sigma0, T, dt, r, q, v, rho, seed=None):
    if seed is not None:
        np.random.seed(seed)

    S = np.zeros(T + 1)
    sigma = np.zeros(T + 1)

    S[0] = S0
    sigma[0] = sigma0

    for t in range(T):
        z1, z2 = correlated_normals(rho)
        S[t+1], sigma[t+1] = simulate_sabr_step(
            S[t], sigma[t], dt, r, q, v, rho, z1[0], z2[0]
        )

    return S, sigma


Approximation SABR de Hagan pour la volatilité implicite.

$$
\sigma_{\text{imp}} =
\begin{cases}
\sigma_0 B, & F = K \\
\sigma_0 B \dfrac{\phi}{\chi}, & F \neq K
\end{cases}
$$


In [28]:
def sabr_implied_vol(S, K, T, sigma0, v, rho, r, q):
    F = S * np.exp((r - q) * T)

    B = 1 + ((rho * v * sigma0) / 4 + (2 - 3 * rho**2) * v**2 / 24) * T

    if np.isclose(F, K):
        return sigma0 * B

    phi = (v / sigma0) * np.log(F / K)
    chi = np.log((np.sqrt(1 - 2 * rho * phi + phi**2) + phi - rho) / (1 - rho))

    return sigma0 * B * phi / chi


Prix d’un call européen (Black--Scholes).

$$
V = S e^{-qT} N(d_1) - K e^{-rT} N(d_2)
$$


In [29]:
def bs_price_call(S, K, T, r, q, sigma):
    if T <= 0:
        return max(S - K, 0)

    d1 = (np.log(S / K) + (r - q + 0.5 * sigma**2) * T) / (sigma * np.sqrt(T))
    d2 = d1 - sigma * np.sqrt(T)

    price = S * np.exp(-q * T) * norm.cdf(d1) - K * np.exp(-r * T) * norm.cdf(d2)
    return price


Delta d’un call européen.

$$
\Delta = e^{-qT} N(d_1)
$$
Gamma d’un call européen.

$$
\Gamma = \frac{e^{-qT} N'(d_1)}{S \sigma \sqrt{T}}
$$

Vega d’un call européen.

$$
\mathcal V = S e^{-qT} \sqrt{T} N'(d_1)
$$


In [30]:
def bs_delta_call(S, K, T, r, q, sigma):
    if T <= 0:
        return 1.0 if S > K else 0.0

    d1 = (np.log(S / K) + (r - q + 0.5 * sigma**2) * T) / (sigma * np.sqrt(T))
    return np.exp(-q * T) * norm.cdf(d1)

def bs_gamma_call(S, K, T, r, q, sigma):
    if T <= 0:
        return 0.0

    d1 = (np.log(S / K) + (r - q + 0.5 * sigma**2) * T) / (sigma * np.sqrt(T))
    return np.exp(-q * T) * norm.pdf(d1) / (S * sigma * np.sqrt(T))

def bs_vega_call(S, K, T, r, q, sigma):
    if T <= 0:
        return 0.0

    d1 = (np.log(S / K) + (r - q + 0.5 * sigma**2) * T) / (sigma * np.sqrt(T))
    return S * np.exp(-q * T) * np.sqrt(T) * norm.pdf(d1)


In [31]:
def price_and_greeks_call(S, K, T, r, q, sigma0, v, rho):
    sigma_imp = sabr_implied_vol(S, K, T, sigma0, v, rho, r, q)

    price = bs_price_call(S, K, T, r, q, sigma_imp)
    delta = bs_delta_call(S, K, T, r, q, sigma_imp)
    gamma = bs_gamma_call(S, K, T, r, q, sigma_imp)
    vega = bs_vega_call(S, K, T, r, q, sigma_imp)

    return {
        "price": price,
        "delta": delta,
        "gamma": gamma,
        "vega": vega,
        "sigma_imp": sigma_imp
    }


Quelques plots pour vérifier le comportement en fct du temps sur 1 an de trading days

In [32]:
RUN_TEST=False

if RUN_TEST:

    # paramètres test
    S0 = 100.0
    sigma0 = 0.2
    r = 0.01
    q = 0.0
    v = 0.5
    rho = -0.3

    T_days = 252
    dt = 1/252
    T_years = T_days * dt

    K = S0  # ATM-forward approx

    # simulation SABR
    S_path, sigma_path = simulate_sabr_path(
        S0=S0,
        sigma0=sigma0,
        T=T_days,
        dt=dt,
        r=r,
        q=q,
        v=v,
        rho=rho,
        seed=42
    )

    # containers
    sigma_imp_path = []
    delta_path = []
    gamma_path = []
    vega_path = []

    for t in range(T_days + 1):
        tau = max(T_years - t * dt, 1e-9)

        res = price_and_greeks_call(
            S=S_path[t],
            K=K,
            T=tau,
            r=r,
            q=q,
            sigma0=sigma_path[t],
            v=v,
            rho=rho
        )

        sigma_imp_path.append(res["sigma_imp"])
        delta_path.append(res["delta"])
        gamma_path.append(res["gamma"])
        vega_path.append(res["vega"])

    # plots
    fig, axs = plt.subplots(3, 2, figsize=(12, 10))
    axs = axs.flatten()

    axs[0].plot(S_path)
    axs[0].set_title("Spot $S_t$")

    axs[1].plot(sigma_path, label=r"Vol instantanée $\sigma_t$")
    axs[1].plot(sigma_imp_path, label=r"Vol implicite $\sigma_{\mathrm{imp}}$ (ATM)")
    axs[1].set_title("Volatilité : instantanée vs implicite")
    axs[1].legend()
    axs[1].grid(True)


    axs[2].plot(delta_path)
    axs[2].set_title("Delta (call ATM)")

    axs[3].plot(gamma_path)
    axs[3].set_title("Gamma (call ATM)")

    axs[4].plot(vega_path)
    axs[4].set_title("Vega (call ATM)")

    for ax in axs:
        ax.grid(True)

    plt.tight_layout()
    plt.show()

Quelques plots pour vérifier le comportement  des greeks en fct du spot sur 1 mois de trading days

In [33]:
if RUN_TEST:

    # paramètres fixes
    K = 100.0
    r = 0.01
    q = 0.0
    sigma0 = 0.2
    v = 0.5
    rho = -0.3

    tau = 30 / 252  # maturité fixée (30 jours)

    # grille de spot
    S_grid = np.linspace(60, 140, 200)

    delta_vals = []
    gamma_vals = []
    vega_vals = []

    for S in S_grid:
        res = price_and_greeks_call(
            S=S,
            K=K,
            T=tau,
            r=r,
            q=q,
            sigma0=sigma0,
            v=v,
            rho=rho
        )
        delta_vals.append(res["delta"])
        gamma_vals.append(res["gamma"])
        vega_vals.append(res["vega"])

    # plots
    fig, axs = plt.subplots(3, 1, figsize=(8, 10), sharex=True)

    axs[0].plot(S_grid, delta_vals)
    axs[0].set_title("Delta vs Spot")
    axs[0].set_ylabel("Delta")
    axs[0].grid(True)

    axs[1].plot(S_grid, gamma_vals)
    axs[1].set_title("Gamma vs Spot")
    axs[1].set_ylabel("Gamma")
    axs[1].grid(True)

    axs[2].plot(S_grid, vega_vals)
    axs[2].set_title("Vega vs Spot")
    axs[2].set_xlabel("Spot $S$")
    axs[2].set_ylabel("Vega")
    axs[2].grid(True)

    plt.tight_layout()
    plt.show()